hello
os library only imported to find the current working folder to relocate the DB file.
SQLalchemy is imported to make SQL queries from the database containing the simulation results.
We are also importing Pandas to read the result of SQL queries into Dataframes for further analysis.

In [ ]:
import os 
import pandas as pd
from sqlalchemy import create_engine

The results database that we are going to read and analyze data from is created by the other project in my account: [Feeder Power Loss Simulation](https://github.com/pooya-bagheri/Feeder-PowerLoss-Simulation)

In [3]:
DBfilePath=os.path.abspath('')+'//'+'SimResultsIEEE123NodesCase2.db'
DB = create_engine('sqlite:///'+DBfilePath, echo=False)   

Database includes data for 30 simulated days. We don't want to explode the memory.So, Let's just focus on the first day read first day's Ploss data and voltages for one node:

In [16]:
Ploss=pd.read_sql_query('Select * from Losses',con=DB)
Ploss.head()

,InstantID,Ploss
0,1,25.598380
1,2,25.351119
2,3,25.104456
3,4,24.859135
4,5,24.615373


In [9]:
Ploss=pd.read_sql_query('''
select Instants.InstantID,Losses.Ploss
from Instants join Losses on Instants.InstantID=Losses.InstantID
where Instants.Day==1''',con=DB)
Ploss.head()
Ploss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440 entries, 0 to 1439
Data columns (total 2 columns):
InstantID    1440 non-null int64
Ploss        1440 non-null float64
dtypes: float64(1), int64(1)
memory usage: 22.6 KB


In [17]:
Voltages=pd.read_sql_query('''
select Instants.InstantID,Voltages.NodeID,Voltages.Vmag
from Instants join Voltages on Instants.InstantID=Voltages.InstantID
where Instants.Day==1''',con=DB)
Voltages.head()

,InstantID,NodeID,Vmag
0,1,1,1.0000
1,1,2,1.0125
2,1,3,1.0125
3,1,4,1.0069
4,1,5,1.0132


In [9]:
print(Ploss.shape)

(44640, 2)


In [18]:
print(Voltages)

        InstantID  NodeID     Vmag
0               1       1  1.00000
1               1       2  1.01250
2               1       3  1.01250
3               1       4  1.00690
4               1       5  1.01320
5               1       6  1.00810
6               1       7  1.00280
7               1       8  1.00760
8               1       9  1.00730
9               1      10  1.00680
10              1      11  1.00020
11              1      12  1.01400
12              1      13  0.99940
13              1      14  0.99676
14              1      15  1.00560
15              1      16  1.00430
16              1      17  1.00370
17              1      18  0.99224
18              1      19  1.00380
19              1      20  1.00410
20              1      21  1.00350
21              1      22  1.00270
22              1      23  1.00340
23              1      24  0.99163
24              1      25  0.99187
25              1      26  0.99153
26              1      27  1.01240
27              1   